In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

In [3]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [4]:
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [5]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

,CategoryID,CategoryName,Description
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an..."
2,3,Confections,"Desserts, candies, and sweet breads"
3,4,Dairy Products,Cheeses
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal"
5,6,Meat/Poultry,Prepared meats
6,7,Produce,Dried fruit and bean curd
7,8,Seafood,Seaweed and fish


In [7]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [11]:
tasks+=1
pd.read_sql(
    '''
    SELECT * FROM Customers WHERE Country='Germany' OR Country='France' OR City='Madrid'
    ''',
    con,
)

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country
0,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209,Germany
1,6,Blauer See Delikatessen,Hanna Moos,Forsterstr. 57,Mannheim,68306,Germany
2,7,Blondel père et fils,Frédérique Citeaux,"24, place Kléber",Strasbourg,67000,France
3,8,Bólido Comidas preparadas,Martín Sommer,"C/ Araquil, 67",Madrid,28023,Spain
4,9,Bon app',Laurence Lebihans,"12, rue des Bouchers",Marseille,13008,France
5,17,Drachenblut Delikatessend,Sven Ottlieb,Walserweg 21,Aachen,52066,Germany
6,18,Du monde entier,Janine Labrune,"67, rue des Cinquante Otages",Nantes,44000,France
7,22,FISSA Fabrica Inter. Salchichas S.A.,Diego Roel,"C/ Moralzarzal, 86",Madrid,28034,Spain
8,23,Folies gourmandes,Martine Rancé,"184, chaussée de Tournai",Lille,59000,France
9,25,Frankenversand,Peter Franken,Berliner Platz 43,München,80805,Germany


2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [12]:
tasks+=1
pd.read_sql(
    '''
    SELECT Country, COUNT(*) as 'Number of Customers'
    FROM Customers
    GROUP BY Country
    ORDER BY COUNT(*) DESC
    LIMIT 3
    ''',
    con,
)

,Country,Number of Customers
0,USA,13
1,Germany,11
2,France,11


3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [16]:
tasks+=1
pd.read_sql(
    '''
    SELECT Shippers.ShipperName, Orders.OrderDate
    FROM Orders
    JOIN Shippers
    ON Orders.ShipperID = Shippers.ShipperID
    ORDER BY Orders.OrderDate
    LIMIT 1 OFFSET 9
    ''',
    con,
)

,ShipperName,OrderDate
0,Federal Shipping,02.10.1996


4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [22]:
tasks+=1
pd.read_sql(
    '''
    WITH OrderTotal AS (
    SELECT OrderID, SUM(Quantity*Price) AS Total
    FROM OrderDetails
    JOIN Products
    ON OrderDetails.ProductID = Products.ProductID
    GROUP BY OrderID
    )
    SELECT Products.ProductName, Products.Price
    FROM OrderTotal
    JOIN OrderDetails
    ON OrderTotal.OrderID = OrderDetails.OrderID
    JOIN Products
    ON OrderDetails.ProductID = Products.ProductID
    WHERE OrderTotal.Total = (SELECT MAX(Total) FROM OrderTotal)
    ''',
    con,
)

,ProductName,Price
0,Sir Rodney's Marmalade,81
1,Côte de Blaye,263.5
2,Camembert Pierrot,34
3,Mozzarella di Giovanni,34.8


5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [23]:
tasks+=1
pd.read_sql(
    '''
    SELECT ProductName, SUM(Quantity) as 'Total Quantity'
    FROM OrderDetails
    JOIN Products
    ON OrderDetails.ProductID = Products.ProductID
    GROUP BY ProductName
    ORDER BY SUM(Quantity) DESC
    LIMIT 1
    ''',
    con,
)

,ProductName,Total Quantity
0,Gorgonzola Telino,458


6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [26]:
tasks+=1
pd.read_sql(
    '''
    WITH SupplierOrders AS (
    SELECT SupplierID, COUNT(DISTINCT OrderID) as 'NumberOfOrders'
    FROM Products
    JOIN OrderDetails
    ON Products.ProductID = OrderDetails.ProductID
    GROUP BY SupplierID
    )
    SELECT SupplierName, Country, ContactName, Phone, NumberOfOrders
    FROM Suppliers
    JOIN SupplierOrders
    ON Suppliers.SupplierID = SupplierOrders.SupplierID
    ORDER BY NumberOfOrders DESC
    LIMIT 5
    ''',
    con,
)

,SupplierName,Country,ContactName,Phone,NumberOfOrders
0,"Pavlova, Ltd.",Australia,Ian Devling,(03) 444-2343,37
1,Plutzer Lebensmittelgroßmärkte AG,Germany,Martin Bein,(069) 992755,37
2,"Specialty Biscuits, Ltd.",UK,Peter Wilson,(161) 555-4448,33
3,Norske Meierier,Norway,Beate Vileid,(0)2-953010,30
4,Formaggi Fortini s.r.l.,Italy,Elio Rossi,(0544) 60323,27


7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [ ]:
tasks+=1
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [ ]:
tasks+=1
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [ ]:
tasks+=1
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

10. Сколько банок крабового мяса всего было заказано.

In [ ]:
tasks+=1
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

In [ ]:
if tasks==10:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')